In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
sitemap_url = "https://wuzzuf.net/sitemap-job-1.xml"
response = requests.get(sitemap_url)

# Parse the XML
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    urls = [loc.text for loc in soup.find_all("loc")]
else:
    print("Failed to fetch the sitemap.")
    urls = []


In [3]:
title = []
working_hours = []
working_place = []
company = []
location = []
post_date = []
num_of_applicants = []
num_of_positions = []
experience = []
career_level = []
education = []
salary = []
job_category = []
skills = []

In [4]:
driver = webdriver.Firefox()

In [5]:
for index, url in enumerate(urls[:2000]):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//section/div/h1"))
        )

        def extract_element(xpath, index=0, multiple=False):
            try:
                elements = driver.find_elements(By.XPATH, xpath)
                if multiple:
                    return [el.text for el in elements] if elements else None
                return elements[index].text if elements else None
            except Exception as e:
                print(f"Error extracting {xpath} from {url}: {str(e)}")
                return None

        # Extract job details
        title.append(extract_element("//section/div/h1"))
        working_hours.append(extract_element("//section/div/div/div/a/span"))
        working_place.append(extract_element("//section/div/div/a/span"))
        company.append(extract_element("//strong/div/a", 1))
        location.append(extract_element("//strong", 1).split('-')[-1].strip() if extract_element("//strong", 1) else None)
        post_date.append(extract_element("//section/div/span"))
        num_of_applicants.append(extract_element("//section/div/div/div/strong"))
        num_of_positions.append(extract_element("//div/div/span/span", 1))
        experience.append(extract_element("//main//section/div/span/span"))
        career_level.append(extract_element("//main//section/div/span/span", 1))
        education.append(extract_element("//main//section/div/span/span", 2))
        salary.append(extract_element("//main//section/div/span/span", 3))
        job_category.append(extract_element("//div/ul/li/a/span"))
        skills.append(extract_element("//div/a/span/span/span", multiple=True))
    
        # Save every 200 rows
        if (index + 1) % 200 == 0:
            jobs_df = pd.DataFrame({
                'Title': title,
                'Working_Hours': working_hours,
                'Working_Place': working_place,
                'Company': company,
                'Location': location,
                'Post_Date': post_date,
                'Number_of_Applicants': num_of_applicants,
                'Number_of_Positions': num_of_positions,
                'Experience': experience,
                'Career_Level': career_level,
                'Education': education,
                'Salary': salary,
                'Job_Category': job_category,
                'Skills': skills
            })
            jobs_df.to_csv("wuzzuf_jobs_backup.csv", index=False)
            print(f"Saved {index + 1} rows to wuzzuf_jobs_backup.csv")
    
    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")

# Final save
jobs_df = pd.DataFrame({
    'Title': title,
    'Working_Hours': working_hours,
    'Working_Place': working_place,
    'Company': company,
    'Location': location,
    'Post_Date': post_date,
    'Number_of_Applicants': num_of_applicants,
    'Number_of_Positions': num_of_positions,
    'Experience': experience,
    'Career_Level': career_level,
    'Education': education,
    'Salary': salary,
    'Job_Category': job_category,
    'Skills': skills
})

jobs_df.to_csv("wuzzuf_jobs_final.csv", index=False)
print("Final data saved to wuzzuf_jobs_final.csv")

# Close the browser
driver.quit()

Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/swGt1fw25yHL-Senior-Property-Consultant-Westridge-Giza-Egypt: list index out of range
Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/Z9m2aA8HxUAt-Sales-Advisor-Dimension-Cairo-Egypt: list index out of range
Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/nNDnWk1jMQ8a-Sales-business-development-Nano-Shield-Cairo-Egypt: list index out of range
Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/g/1XDYttSzKSTz-Brand-Specialist-Amazon-Vendor-Services-Saudi-National-Afaq-Q-Tech-General-Trading-Riyadh-Saudi-Arabia: list index out of range
Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/RXusn3fhdJHI-Telesales-Representative---UK-Account-ONLY-Global-Travel-Association-Cairo-Egypt: list index out of range
Error extracting //div/div/span/span from https://wuzzuf.net/jobs/p/CEPfvasatZS3-Spanish-English-Customer-Service-Agent-VXI-Cairo-Egypt: list index o